In [2]:
import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup
print('Libraries imported.')

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.
Libraries imported.


In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})
#print (My_table)

table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitb = soup.findAll("table", table_classes)
#wikitb

In [4]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toront are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toront are 43.653963, -79.387207.


In [5]:
# define the dataframe columns
column_names = ['PostCode', 'Borough', 'Neighbourhood'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

column1 = []
column2 = []
column3 = []

for row in wikitb[0].findAll('tr'):
    cells = row.findAll(['th','td'])
    #print(cells)
    for c in cells[0].findAll(text=True):
        #print (c)
        column1.append(c)
    for c in cells[1].findAll(text=True):
        #print (c)
        column2.append(c)
    #print (cells[2])
    for c in cells[2].findAll(text=True):
        #print (c.strip())
        column3.append(c.strip())
   
column1.pop(0)
column2.pop(0)
column3.pop(0)

neighborhoods['PostCode'] = column1
neighborhoods['Borough'] = column2
#print (len(column1))
#print (len(column2))
#print (len(column3))

column3= [x for x in column3 if x!='']
#neighborhoods['Borough'] = column2
neighborhoods['Neighbourhood'] = column3
#neighborhoods

neighborhoods = neighborhoods[neighborhoods["Borough"]!="Not assigned"]
neighborhoods = neighborhoods.groupby(['PostCode','Borough'], as_index = False)['Neighbourhood'].agg(list)

#neighborhoods['Neighbourhood'] = neighborhoods['Neighbourhood'].map(lambda x: x.lstrip('[').rstrip(']'))
neighborhoods['Neighbourhood'] = neighborhoods["Neighbourhood"].apply(lambda x: ', '.join(x))

line = neighborhoods[neighborhoods["Neighbourhood"]=="Not assigned"]
line['Neighbourhood'] = line['Borough']
neighborhoods[neighborhoods["Neighbourhood"]=="Not assigned"] = line

neighborhoods

neighborhoods.shape

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(103, 3)

Assumption: there was only 1 table

Assumption: assuming the Columns' names

Assumption: all original columns had the same size

In [6]:
'''import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    print('HI')
    g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
    #g = geocoder.google('Toronto')
    lat_lng_coords = g.latlng
    print('R')

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
longitude'''

'import geocoder # import geocoder\n\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n    print(\'HI\')\n    g = geocoder.google(\'{}, Toronto, Ontario\'.format("M5G"))\n    #g = geocoder.google(\'Toronto\')\n    lat_lng_coords = g.latlng\n    print(\'R\')\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\nlongitude'

In [7]:
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
neighborhoods.head()

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
neighborhoods.rename(columns={'PostCode':'Postal Code'}, inplace=True)
left = neighborhoods
right = coords
neighborhoods_combined = pd.merge(left, right, on='Postal Code')
neighborhoods_combined.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
